In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::792977523885:role/AWSGlueServiceRole-Lab4
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 55bbc716-e903-4f28-8948-77eb0c4865a9
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session 55bbc716-e903-4f28-8948-77eb0c4865a9 to get into ready status...
Session 55bbc716-e903-4f28-8948-77eb0c4865a9 has been created.




- Na célula a seguir, adicione o código necessário para ler o conteúdo do arquivo _nomes.csv_ para um dataframe Spark.



In [6]:
df_nomes = spark.read.format("csv") \
        .option("header", True) \
        .option("delimiter", ',') \
        .option("inferSchema", True) \
        .load("s3://labxglue/nomes.csv")

- Imprima o esquema do dataframe

In [7]:
df_nomes.printSchema()
df_nomes.show()

root
 |-- nome: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- ano: integer (nullable = true)

+---------+----+-----+----+
|     nome|sexo|total| ano|
+---------+----+-----+----+
| Jennifer|   F|54336|1983|
|  Jessica|   F|45278|1983|
|   Amanda|   F|33752|1983|
|   Ashley|   F|33292|1983|
|    Sarah|   F|27228|1983|
|  Melissa|   F|23472|1983|
|   Nicole|   F|22392|1983|
|Stephanie|   F|22323|1983|
|  Heather|   F|20749|1983|
|Elizabeth|   F|19838|1983|
|  Crystal|   F|17904|1983|
|      Amy|   F|17095|1983|
| Michelle|   F|16828|1983|
|  Tiffany|   F|15960|1983|
| Kimberly|   F|15374|1983|
|Christina|   F|15359|1983|
|    Amber|   F|14886|1983|
|     Erin|   F|14835|1983|
|  Rebecca|   F|14711|1983|
|   Rachel|   F|14588|1983|
+---------+----+-----+----+
only showing top 20 rows


- Na próxima célula, escreva o código necessário para alterar a caixa dos valores da coluna _nome_ para MAIÚSCULO

In [8]:
from pyspark.sql.functions import upper, col
df_upper = df_nomes.withColumn('nome', upper(col('nome'))).show()

+---------+----+-----+----+
|     nome|sexo|total| ano|
+---------+----+-----+----+
| JENNIFER|   F|54336|1983|
|  JESSICA|   F|45278|1983|
|   AMANDA|   F|33752|1983|
|   ASHLEY|   F|33292|1983|
|    SARAH|   F|27228|1983|
|  MELISSA|   F|23472|1983|
|   NICOLE|   F|22392|1983|
|STEPHANIE|   F|22323|1983|
|  HEATHER|   F|20749|1983|
|ELIZABETH|   F|19838|1983|
|  CRYSTAL|   F|17904|1983|
|      AMY|   F|17095|1983|
| MICHELLE|   F|16828|1983|
|  TIFFANY|   F|15960|1983|
| KIMBERLY|   F|15374|1983|
|CHRISTINA|   F|15359|1983|
|    AMBER|   F|14886|1983|
|     ERIN|   F|14835|1983|
|  REBECCA|   F|14711|1983|
|   RACHEL|   F|14588|1983|
+---------+----+-----+----+
only showing top 20 rows


- Apresente a contagem de linhas contidas no dataframe

In [12]:
df_nomes.count()

1825433


- Agora, apresente a contagem de nomes, agrupando os dados do dataframe pelas colunas _ano_ e _sexo_. Ordene os dados de modo que o ano mais recente aparece como primeiro registro do dataframe.

In [48]:
agrup_nomes = df_nomes.groupBy('ano', 'sexo').count().orderBy('ano', ascending=False) 
resultado = agrup_nomes.select('ano', 'sexo', 'count').show()



+----+----+-----+
| ano|sexo|count|
+----+----+-----+
|2014|   F|19067|
|2014|   M|13977|
|2013|   F|19191|
|2013|   M|14012|
|2012|   F|19468|
|2012|   M|14216|
|2011|   F|19540|
|2011|   M|14329|
|2010|   M|14241|
|2010|   F|19800|
|2009|   M|14519|
|2009|   F|20165|
|2008|   F|20439|
|2008|   M|14606|
|2007|   M|14383|
|2007|   F|20548|
|2006|   M|14026|
|2006|   F|20043|
|2005|   F|19175|
|2005|   M|13358|
+----+----+-----+
only showing top 20 rows


- Qual foi o nome feminino com mais registro e em que ano?

In [79]:
from pyspark.sql.functions import desc, sum
nome_f = df_nomes.where(df_nomes.sexo == 'F')
nome_c = nome_f.groupBy('nome', 'ano').agg(sum('total').alias('total_nome')).orderBy('total_nome', ascending=False)
nome_max = nome_c.first()
print(nome_max)



Row(nome='Linda', ano=1947, total_nome=99680)


- Qual foi o nome masculino com mais registro e em que ano?

In [80]:
from pyspark.sql.functions import desc, sum
nome_m = df_nomes.where(df_nomes.sexo == 'M')
nome_c = nome_m.groupBy('nome', 'ano').agg(sum('total').alias('total_nome')).orderBy('total_nome', ascending=False)
nome_max = nome_c.first()
print(nome_max)

Row(nome='James', ano=1947, total_nome=94755)


- Apresente o total de registros (masculinos e femininos) para cada ano presente no dataframe. Considere apenas as primeiras 10 linhas, ordenadas pelo ano, de forma crescente.

In [84]:
nome_c = df_nomes.groupBy('sexo', 'ano').agg(sum('total').alias('total_ano')).orderBy('ano', descending=False).limit(10)
nome_max = nome_c.show()

+----+----+---------+
|sexo| ano|total_ano|
+----+----+---------+
|   M|1880|   110491|
|   F|1880|    90993|
|   F|1881|    91954|
|   M|1881|   100745|
|   M|1882|   113688|
|   F|1882|   107850|
|   M|1883|   104629|
|   F|1883|   112321|
|   M|1884|   114445|
|   F|1884|   129022|
+----+----+---------+


- Escreva o conteúdo do dataframe com os valores de nome em maiúsculo no S3. Atenção aos requisitos:
  - A gravação deve ocorrer no subdiretório _frequencia_registro_nomes_eua_ do path s3://\<BUCKET\>/lab-glue/
  - O formato deve ser JSON
  - O particionamento deverá ser realizado pelas colunas _sexo_ e _ano_ (nesta ordem)

In [104]:
import boto3
import pandas as pd
from awsglue.dynamicframe import DynamicFrame



s3_client = boto3.client('s3')
response = s3_client.get_object(Bucket="labxglue",Key="nomes.csv")
file = pd.read_csv(response["Body"])
glue_context = GlueContext(SparkContext.getOrCreate())

df_upper = df_nomes.withColumn('nome', upper(col('nome')))
output_dir = f's3://aws-glue-assets-792977523885-us-east-1/'
glue_context.write_dynamic_frame.from_options(
    frame=DynamicFrame.fromDF(df_upper, glueContext, "extracao_final"),
    connection_type="s3",
    connection_options={
        "path": output_dir,
        "partitionKeys": ["sexo", "ano"]
    },
    format="json"
)


In [105]:
job.commit()